Author: Shota Ichitaku

In [1]:
import numpy as np
import pandas as pd

In [2]:
!pip install openai
!pip install langchain
!pip install langchain-chroma
!pip install -U langchain-community
!pip install tiktoken
!pip install unstructured
!pip install --upgrade --force-reinstall pandas scipy numba


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 611.1/611.1 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 54.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 91.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 8.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 5.2 MB/s eta 0:00:

In [3]:
import os
import openai
import sys
sys.path.append('../..')

os.environ['OPENAI_API_KEY'] =  ''

In [4]:
# Uncomment if you aare using colab. Now we connect the colab notebook to our drive to access the data.
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [5]:
%cd /content/gdrive/MyDrive/'Colab Notebooks'/MF815/NLP/'NLP_app '

/content/gdrive/MyDrive/Colab Notebooks/MF815/NLP/NLP_app 


In [6]:
!ls

df_RAG2_5.csv		   MutualFundSummary2
df_RAG2.csv		   MutualFundSummary2_5
df_RAG3_5.csv		   MutualFundSummary3
df_RAG3.csv		   MutualFundSummary3_5
df_RAG4.csv		   MutualFundSummary4
df_RAG_all.csv		   MutualFundSummary4_5
df_RAG.csv		   MutualFundSummary5
docs			   MutualFundSummary6
FundArticlesHealthy.csv    MutualFundSummary7
FundArticlesUnhealthy.csv  MutualFundSummary8
MutualFundLabels.csv	   MutualFundSummary9
MutualFundSummary	   NLP_Deep_Learning_Questrom_WS_03_2020.ipynb
MutualFundSummary1	   word2vec_perso.txt
MutualFundSummary10


In [7]:
from IPython.display import HTML

In [8]:
# Progress bar
def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

# read the repo in PATH and append the texts in a list
def get_data(PATH):
    list_dir = os.listdir(PATH)
    texts = []
    fund_names = []
    out = display(progress(0, len(list_dir)-1), display_id=True)
    for ii, filename in enumerate(list_dir) :
        with open(PATH+'/'+filename, 'r', encoding="utf8") as f :
            txt = f.read()
            try :
                txt_split = txt.split('<head_breaker>')
                summary = txt_split[1].strip()
                fund_name = txt_split[0].strip()
            except :
                summary = txt
                fund_name = ''
        texts.append(summary)
        fund_names.append(fund_name)
        out.update(progress(ii, len(list_dir)-1))
    return fund_names, texts

In [9]:
# Get the summaries

fund_names, summaries = get_data('MutualFundSummary')

In [10]:
#fund_names

In [68]:
print(len(fund_names))

545


In [11]:
df_extraction = pd.DataFrame({'fund_name' : fund_names})
df_label = pd.read_csv('MutualFundLabels.csv')
df = df_label.merge(df_extraction, on='fund_name', how='inner')
df_label = df_label.rename(columns={'Leverage?': 'Derivatives_label'})
df_label

,id,fund_name,Performance fee?,Ivestment Strategy,Derivatives_label,Portfolio composition,Concentration
0,0000051931-18-000151,American Funds College 2018 Fund,NaN,Balanced Fund (Low Risk),Yes,Investment grade securities,Diversified
1,0000051931-18-000151,American Funds College 2021 Fund,NaN,Balanced Fund (Low Risk),Yes,Investment grade securities,Diversified
2,0000051931-18-000151,American Funds College 2024 Fund,NaN,Balanced Fund (Low Risk),Yes,Investment grade securities,Diversified
3,0000051931-18-000151,American Funds College 2027 Fund,NaN,Balanced Fund (Low Risk),Yes,Investment grade securities,Diversified
4,0000051931-18-000151,American Funds College 2030 Fund,NaN,Balanced Fund (Low Risk),Yes,Investment grade securities,Diversified
...,...,...,...,...,...,...,...
462,0001710607-18-000172,American Century Diversified Corporate Bond ETF,NaN,Fixed Income Long Only (Low Risk),Yes,Investment grade securities,Diversified
463,0001710607-18-000172,American Century Diversified Municipal Bond ETF,NaN,Fixed Income Long Only (Low Risk),No,Investment grade securities,Diversified
464,0001710607-18-000172,American Century Quality Diversified Internati...,NaN,Equity Long Only (Low Risk),Yes,Sub-investment grade securities or emerging ma...,Diversified
465,0001710607-18-000172,American Century STOXX U.S. Quality Growth ETF,NaN,Equity Long Only (Low Risk),Yes,Listed Equities,Diversified


In [12]:
df

,id,fund_name,Performance fee?,Ivestment Strategy,Leverage?,Portfolio composition,Concentration
0,0000051931-18-000151,American Funds College 2018 Fund,NaN,Balanced Fund (Low Risk),Yes,Investment grade securities,Diversified
1,0000051931-18-000151,American Funds College 2021 Fund,NaN,Balanced Fund (Low Risk),Yes,Investment grade securities,Diversified
2,0000051931-18-000151,American Funds College 2024 Fund,NaN,Balanced Fund (Low Risk),Yes,Investment grade securities,Diversified
3,0000051931-18-000151,American Funds College 2027 Fund,NaN,Balanced Fund (Low Risk),Yes,Investment grade securities,Diversified
4,0000051931-18-000151,American Funds College 2030 Fund,NaN,Balanced Fund (Low Risk),Yes,Investment grade securities,Diversified
...,...,...,...,...,...,...,...
461,0001710607-18-000172,American Century Diversified Corporate Bond ETF,NaN,Fixed Income Long Only (Low Risk),Yes,Investment grade securities,Diversified
462,0001710607-18-000172,American Century Diversified Municipal Bond ETF,NaN,Fixed Income Long Only (Low Risk),No,Investment grade securities,Diversified
463,0001710607-18-000172,American Century Quality Diversified Internati...,NaN,Equity Long Only (Low Risk),Yes,Sub-investment grade securities or emerging ma...,Diversified
464,0001710607-18-000172,American Century STOXX U.S. Quality Growth ETF,NaN,Equity Long Only (Low Risk),Yes,Listed Equities,Diversified


## Ask what investment strategy do they take

In [70]:
from langchain.document_loaders import DirectoryLoader

# Load documents from a directory
loader = DirectoryLoader(path="./MutualFundSummary", glob="*.txt")

# Load the text data from the directory
documents = loader.load()

In [71]:
#print(len(documents))

In [72]:
#document = documents[0]

In [73]:
#print(document)

In [78]:
from langchain.text_splitter import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain_chroma import Chroma
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings()
persist_directory = './docs/chroma'

#llm_name = "gpt-3.5-turbo"
llm_name = "gpt-4o-mini"
print(llm_name)

from langchain.chat_models import ChatOpenAI
llm = ChatOpenAI(model_name=llm_name, temperature=0)
from langchain.chains import RetrievalQA

gpt-4o-mini


In [79]:
r_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", "(?<=\. )", " "]
)

In [80]:
#splits=r_splitter.split_documents(documents)
ii = 4

splits=r_splitter.split_documents([documents[ii]])

fund_name = fund_names[ii]

persist_directory = f"./docs/chroma/{fund_name}"

vectordb = Chroma.from_documents(
  documents=splits,
  embedding=embedding,
  persist_directory=persist_directory
)

vectordb.persist()

print(len(vectordb))

341


In [82]:
from langchain.prompts import PromptTemplate

In [83]:
# Build prompt
template = """Use the following pieces of context to answer the question at the end. Please answer only in following terms: Balanced Fund (LowRisk) or Fixed Income Long Only (Low Risk) or
Equity Long Only (Low Risk) or Long Short Funds (High Risk) or Commodities Fund (Low Risk) . If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Helpful Answer:"""
QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

In [85]:
qa_chain = RetrievalQA.from_chain_type(
  llm,
  retriever=vectordb.as_retriever(),
  return_source_documents=True,
  chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
)

In [86]:
#fund_name

In [87]:

#question = f"What category of fund can {fund_name} be classified as? "
#question = f"Who are the portfolio managers of Ultra-Short Bond Fund?"

#result = qa_chain({"query": question})

In [88]:
#result

In [89]:
#print(result["source_documents"][0].metadata)

In [90]:
df_RAG = pd.DataFrame(fund_names, columns=['fund_name'])
df_RAG['Predicted investment strategy'] = np.nan
df_RAG['investment strategy evidence'] = np.nan
df_RAG['source meta data'] = np.nan

In [91]:
df_RAG

,fund_name,Predicted investment strategy,investment strategy evidence,source meta data
0,AB Arizona Portfolio,NaN,NaN,NaN
1,AB Asia ex-Japan Equity Portfolio,NaN,NaN,NaN
2,AB Credit Long/Short Portfolio,NaN,NaN,NaN
3,AB Balanced Wealth Strategy Portfolio,NaN,NaN,NaN
4,AB Bond Inflation Strategy,NaN,NaN,NaN
...,...,...,...,...
540,Variable Portfolio - Victory Sycamore Established Value Fund,NaN,NaN,NaN
541,Variable Portfolio - TCW Core Plus Bond Fund,NaN,NaN,NaN
542,Western Asset Intermediate Maturity New York Municipals Fund,NaN,NaN,NaN
543,Western Asset Massachusetts Municipals Fund,NaN,NaN,NaN


In [43]:
import time

In [92]:

for ii in range(len(df_RAG)):
  #print(ii)

  fund_name = df_RAG.iloc[ii]['fund_name']

  print(fund_name)

  splits=r_splitter.split_documents([documents[ii]])

  persist_directory = f"./docs/chroma/{fund_name}"

  vectordb = Chroma.from_documents(
    documents=splits,
    embedding=embedding,
    persist_directory=persist_directory
  )

  vectordb.persist()

  print(len(vectordb))

  # Build prompt
  template = """Use the following pieces of context to answer the question at the end. Please answer only in following terms: Balanced Fund (LowRisk) or Fixed Income Long Only (Low Risk) or
Equity Long Only (Low Risk) or Long Short Funds (High Risk) or Commodities Fund (Low Risk) . If you don't know the answer, just say that you don't know, don't try to make up an answer.
{context}
Question: {question}
Helpful Answer:"""
  QA_CHAIN_PROMPT = PromptTemplate.from_template(template)

  qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=vectordb.as_retriever(),
    return_source_documents=True,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT}
  )

  question = f"What category of fund can {fund_name} be classified as? "


  result = qa_chain({"query": question})

  #print(result["result"])

  df_RAG.iloc[ii, 1] = result["result"]

  df_RAG.iloc[ii, 2] = result["source_documents"][0].page_content

  df_RAG.iloc[ii, 3] = str(result["source_documents"][0].metadata)

  # --- ADD THE DELAY HERE ---
    # This pause happens *after* all processing for index 'ii' is done,
    # including the API call, before starting the next iteration (ii+1).
  #print(f"--- Finished processing index {ii}. Waiting for 21 seconds before next API call... ---")
  #time.sleep(21) # Pause for 10 seconds
    # --------------------------



AB Arizona Portfolio
297


<ipython-input-92-0973c680d478>:44: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Fixed Income Long Only (Low Risk)' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_RAG.iloc[ii, 1] = result["result"]
<ipython-input-92-0973c680d478>:46: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'AB Arizona Portfolio

<head_breaker>

AB Arizona Portfolio

--------------------------------------------------------------------------------

INVESTMENT OBJECTIVE: The investment objective of the Portfolio is to earn the highest level of current income exempt from both federal income tax and State of Arizona personal income tax that is available without assuming what the Adviser considers to be undue risk.

FEES AND EXPENSES OF THE PORTFOLIO:' has dtype incompatible with float64, please explicitly cas

AB Asia ex-Japan Equity Portfolio
279
AB Credit Long/Short Portfolio
403
AB Balanced Wealth Strategy Portfolio
270
AB Bond Inflation Strategy
372
AB All Market Income Portfolio
203
AB All Market Real Return Portfolio
266
AB Discovery Value Fund
162
AB Dynamic Asset Allocation Portfolio
135
AB FlexFee International Strategic Core Portfolio
104
AB FlexFee Core Opportunities Portfolio
120
AB FlexFee Emerging Markets Growth Portfolio
85
AB Global Risk Allocation-Moderate Portfolio
104
AB FlexFee Large Cap Growth Portfolio
96
AB Global Thematic Growth Portfolio
68
AB Income Fund
124
AB Intermediate Bond Portfolio
128
AB Limited Duration High Income Portfolio
120
AB Massachusetts Portfolio
144
AB Minnesota Portfolio
132
AB Municipal Bond Inflation Strategy
116
AB Tax-Aware Fixed Income Portfolio
124
AB New Jersey Portfolio
136
AB New York Portfolio
140
AB Real Estate Investment Portfolio
136
AB Pennsylvania Portfolio
136
AB Small/Mid Cap Value Portfolio
76
AB Small Cap Value Portfolio
92
ADA

In [93]:
pd.set_option('display.max_colwidth', None)

In [94]:
df_RAG

,fund_name,Predicted investment strategy,investment strategy evidence,source meta data
0,AB Arizona Portfolio,Fixed Income Long Only (Low Risk),AB Arizona Portfolio\n\n<head_breaker>\n\nAB Arizona Portfolio\n\n--------------------------------------------------------------------------------\n\nINVESTMENT OBJECTIVE: The investment objective of the Portfolio is to earn the highest level of current income exempt from both federal income tax and State of Arizona personal income tax that is available without assuming what the Adviser considers to be undue risk.\n\nFEES AND EXPENSES OF THE PORTFOLIO:,{'source': 'MutualFundSummary/AB Arizona Portfolio.txt'}
1,AB Asia ex-Japan Equity Portfolio,Equity Long Only (Low Risk),"AB Asia ex\n\nJapan Equity Portfolio\n\n<head_breaker>\n\nAB ASIA EX\n\nJAPAN EQUITY PORTFOLIO\n\n--------------------------------------------------------------------------------\n\nAT A RECENT MEETING, THE FUND'S BOARD OF DIRECTORS APPROVED THE PROPOSED LIQUIDATION AND TERMINATION OF THE FUND (THE ""LIQUIDATION""). IN CONNECTION WITH THE LIQUIDATION, THE FUND HAS SUSPENDED MOST SALES OF ITS SHARES AND EXCHANGES INTO THE FUND. IN LIMITED CIRCUMSTANCES, SUCH AS SALES TO CERTAIN RETIREMENT PLANS AND SALES MADE THROUGH RETAIL OMNIBUS PLATFORMS, THE FUND WILL CONTINUE TO OFFER ITS SHARES, ALTHOUGH WITHOUT AN INITIAL SALES CHARGE. IN ADDITION, THE FUND HAS SUSPENDED DISTRIBUTION AND/OR SERVICE (RULE 12B-1) FEES AND WAIVED CONTINGENT DEFERRED SALES CHARGES UPON REDEMPTIONS OF THE FUND'S SHARES.\n\nINVESTMENT OBJECTIVE The Fund's investment objective is to seek long-term growth of capital.\n\nFEES AND EXPENSES OF THE FUND",{'source': 'MutualFundSummary/AB Asia ex-Japan Equity Portfolio.txt'}
2,AB Credit Long/Short Portfolio,Long Short Funds (High Risk),"AB Credit Long/Short Portfolio\n\n<head_breaker>\n\nAB CREDIT LONG/SHORT PORTFOLIO\n\n--------------------------------------------------------------------------------\n\nINVESTMENT OBJECTIVE The Fund's investment objective is to seek absolute return over a full market cycle.\n\nFEES AND EXPENSES OF THE FUND\n\nThis table describes the fees and expenses that you may pay if you buy and hold shares of the Fund. You may qualify for sales charge reductions if you and members of your family invest, or agree to invest in the future, at least $100,000 in AB Mutual Funds. More information about these and other discounts is available from your financial intermediary and in Investing in the Funds--Sales Charge Reduction Programs for Class A Shares on page 61 of this Prospectus, in Appendix C--Financial Intermediary Waivers of this Prospectus and in Purchase of Shares--Sales Charge Reduction Programs for Class A Shares on page 126 of the Fund's Statement of Additional Information (""SAI"").",{'source': 'MutualFundSummary/AB Credit LongShort Portfolio.txt'}
3,AB Balanced Wealth Strategy Portfolio,Balanced Fund (Low Risk),"AB Balanced Wealth Strategy Portfolio <head_breaker>\n\nAB VPS BALANCED WEALTH STRATEGY PORTFOLIO --------------------------------------------------------------------------------\n\nINVESTMENT OBJECTIVE The Portfolio's investment objective is to maximize total return consistent with the Adviser's determination of reasonable risk.\n\nFEES AND EXPENSES OF THE PORTFOLIO\n\nThis table describes the fees and expenses that you may pay if you buy and hold shares of the Portfolio. Because the information does not reflect deductions at the separate account level or contract level for any charges that may be incurred under a contract, Contractholders that invest in the Portfolio should refer to the variable contract prospectus for a description of fees and expenses that apply to Contractholders. Inclusion of these charges would increase the fees and expenses provided below.\n\nSHAREHOLDER FEES (fees paid directly from your investment) N/A",{'source': 'MutualFundSummary/AB Balanced Wealth Strategy Portfolio.txt'}
4,AB Bond Inflation Strategy,Fixed Income Long Only (Low Risk),AB Bon

In [95]:
df_RAG.to_csv('df_RAG_all.csv', index=False)

## Create new folder for failed process

In [117]:
fail = df_RAG[33:]
fail

,fund_name,Predicted investment strategy,investment strategy evidence,source meta data
33,Fidelity Series Opportunistic Insights Fund,NaN,NaN,NaN
34,Fidelity Series Short-Term Credit Fund,NaN,NaN,NaN
35,Fidelity Simplicity RMD 2005 Fund,NaN,NaN,NaN
36,Fidelity Simplicity RMD 2020 Fund,NaN,NaN,NaN
37,Fidelity ZERO Extended Market Index Fund,NaN,NaN,NaN
38,Fidelity Simplicity RMD Income Fund,NaN,NaN,NaN
39,Fidelity ZERO Large Cap Index Fund,NaN,NaN,NaN
40,Franklin FTSE Asia ex Japan ETF,NaN,NaN,NaN
41,FOCUSED INTERNATIONAL GROWTH FUND,NaN,NaN,NaN
42,Franklin FTSE Brazil ETF,NaN,NaN,NaN


In [119]:
fund_name = fail['fund_name'].tolist()
fund_name

['Fidelity Series Opportunistic Insights Fund',
 'Fidelity Series Short-Term Credit Fund',
 'Fidelity Simplicity RMD 2005 Fund',
 'Fidelity Simplicity RMD 2020 Fund',
 'Fidelity ZERO Extended Market Index Fund',
 'Fidelity Simplicity RMD Income Fund',
 'Fidelity ZERO Large Cap Index Fund',
 'Franklin FTSE Asia ex Japan ETF',
 'FOCUSED INTERNATIONAL GROWTH FUND',
 'Franklin FTSE Brazil ETF',
 'Franklin FTSE Australia ETF',
 'Franklin FTSE China ETF',
 'Franklin FTSE Europe Hedged ETF',
 'Franklin FTSE Germany ETF',
 'Franklin FTSE Europe ETF',
 'Franklin FTSE Canada ETF',
 'Franklin FTSE France ETF',
 'Franklin FTSE Hong Kong ETF',
 'Franklin FTSE Italy ETF',
 'Franklin FTSE India ETF',
 'Franklin FTSE Japan Hedged ETF',
 'Franklin FTSE Japan ETF']

In [121]:
#add .txt
fund_names_with_txt = [name + ".txt" for name in fund_name]
fund_names_with_txt

['Fidelity Series Opportunistic Insights Fund.txt',
 'Fidelity Series Short-Term Credit Fund.txt',
 'Fidelity Simplicity RMD 2005 Fund.txt',
 'Fidelity Simplicity RMD 2020 Fund.txt',
 'Fidelity ZERO Extended Market Index Fund.txt',
 'Fidelity Simplicity RMD Income Fund.txt',
 'Fidelity ZERO Large Cap Index Fund.txt',
 'Franklin FTSE Asia ex Japan ETF.txt',
 'FOCUSED INTERNATIONAL GROWTH FUND.txt',
 'Franklin FTSE Brazil ETF.txt',
 'Franklin FTSE Australia ETF.txt',
 'Franklin FTSE China ETF.txt',
 'Franklin FTSE Europe Hedged ETF.txt',
 'Franklin FTSE Germany ETF.txt',
 'Franklin FTSE Europe ETF.txt',
 'Franklin FTSE Canada ETF.txt',
 'Franklin FTSE France ETF.txt',
 'Franklin FTSE Hong Kong ETF.txt',
 'Franklin FTSE Italy ETF.txt',
 'Franklin FTSE India ETF.txt',
 'Franklin FTSE Japan Hedged ETF.txt',
 'Franklin FTSE Japan ETF.txt']

In [122]:
len(fund_names_with_txt)

22

In [123]:
%cd /content/gdrive/MyDrive/'Colab Notebooks'/MF815/NLP/'NLP_app '/MutualFundSummary4

/content/gdrive/MyDrive/Colab Notebooks/MF815/NLP/NLP_app /MutualFundSummary4


In [124]:
!mkdir -p MutualFundSummary4_5

In [125]:
#move failed file to new folder
!mv \
'Fidelity Series Opportunistic Insights Fund.txt' \
'Fidelity Series Short-Term Credit Fund.txt' \
'Fidelity Simplicity RMD 2005 Fund.txt' \
'Fidelity Simplicity RMD 2020 Fund.txt' \
'Fidelity ZERO Extended Market Index Fund.txt' \
'Fidelity Simplicity RMD Income Fund.txt' \
'Fidelity ZERO Large Cap Index Fund.txt' \
'Franklin FTSE Asia ex Japan ETF.txt' \
'FOCUSED INTERNATIONAL GROWTH FUND.txt' \
'Franklin FTSE Brazil ETF.txt' \
'Franklin FTSE Australia ETF.txt' \
'Franklin FTSE China ETF.txt' \
'Franklin FTSE Europe Hedged ETF.txt' \
'Franklin FTSE Germany ETF.txt' \
'Franklin FTSE Europe ETF.txt' \
'Franklin FTSE Canada ETF.txt' \
'Franklin FTSE France ETF.txt' \
'Franklin FTSE Hong Kong ETF.txt' \
'Franklin FTSE Italy ETF.txt' \
'Franklin FTSE India ETF.txt' \
'Franklin FTSE Japan Hedged ETF.txt' \
'Franklin FTSE Japan ETF.txt' \
MutualFundSummary4_5/

In [126]:
%cd /content/gdrive/MyDrive/'Colab Notebooks'/MF815/NLP/'NLP_app '

/content/gdrive/MyDrive/Colab Notebooks/MF815/NLP/NLP_app 
